In [1]:
import numpy as np, os
import pandas as pd

# import optuna

# https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/274717 
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

In [2]:
import random    
seed_num = 42
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 10, 4069), (6182,), (1545, 10, 4069), (1545,))

In [ ]:
from tensorflow.keras.losses import BinaryCrossentropy

lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
es = EarlyStopping(monitor="val_loss", patience=5, verbose=1, mode="min", restore_best_weights=True)
        
results = []
print(' Computing LSTM feature importance...')

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

ep = 500
pa = 30
early_stop = EarlyStopping(monitor='val_acc', patience=pa, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= "adam", loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=ep, callbacks=[lr, es], shuffle=False)

# COLS
a = pd.read_csv('total_data.csv')
COLS = list(a['ITEMID'].sort_values().unique())

results = []

# COMPUTE BASELINE (NO SHUFFLE)
from tensorflow.keras.losses import BinaryCrossentropy
bce = BinaryCrossentropy()

preds = lstm.predict(X_test, verbose=0)
baseline_bce = bce(y_test, preds).numpy()
results.append({'feature':'BASELINE','baseline_bce':baseline_bce}) 

for k in tqdm(range(len(COLS))):

    # SHUFFLE FEATURE K
    save_col = X_test[:,:,k].copy()
    np.random.shuffle(X_test[:,:,k])

    # COMPUTE OOF MAE WITH FEATURE K SHUFFLED
    preds = lstm.predict(X_test, verbose=0)
    loss_bce = bce(y_test, preds).numpy()

    results.append({'feature':COLS[k],'bce':bce})
    X_test[:,:,k] = save_col
    
    # DISPLAY LSTM FEATURE IMPORTANCE
    df = pd.DataFrame(results)

 Computing LSTM feature importance...
Epoch 1/500
37/37 [==============================] - 19s 88ms/step - loss: 0.6789 - acc: 0.6040 - val_loss: 0.6652 - val_acc: 0.6177
Epoch 2/500
37/37 [==============================] - 2s 55ms/step - loss: 0.6727 - acc: 0.6065 - val_loss: 0.6595 - val_acc: 0.6177
Epoch 3/500
37/37 [==============================] - 2s 55ms/step - loss: 0.6574 - acc: 0.6137 - val_loss: 0.5639 - val_acc: 0.7225
Epoch 4/500
37/37 [==============================] - 2s 55ms/step - loss: 0.5397 - acc: 0.7385 - val_loss: 0.4977 - val_acc: 0.7697
Epoch 5/500
37/37 [==============================] - 2s 53ms/step - loss: 0.4558 - acc: 0.8038 - val_loss: 0.4967 - val_acc: 0.7781
Epoch 6/500
37/37 [==============================] - 2s 54ms/step - loss: 0.3980 - acc: 0.8417 - val_loss: 0.5468 - val_acc: 0.7613
Epoch 7/500
37/37 [==============================] - 2s 55ms/step - loss: 0.3719 - acc: 0.8443 - val_loss: 0.5191 - val_acc: 0.7684
Epoch 8/500
37/37 [==================

  0%|          | 0/4069 [00:00<?, ?it/s]

In [ ]:
df